# Table Aggregation

In this notebook we aggregate the raw data into tables as used in our figures.

In [1]:
import pandas as pd

def aggregate(df):
    best_of_10 = df.groupby(["sample_id"]).max()
    best_of_10["num_full_match"] = (best_of_10["consistency"] == 1.0)
    res = best_of_10.groupby(["num_nodes", "num_networks"]).mean()
    res["num_full_match"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_full_match"]
    return res

def aggregate_3pred(df, consistency_column="overall"):
    best_of_10 = df.groupby(["sample_id"]).max()
    best_of_10["num_full_match"] = (best_of_10["overall"] == 1.0)
    best_of_10["num_close_match"] = (best_of_10["overall"] > 0.95)
    best_of_10["num_close_match90"] = (best_of_10["overall"] > 0.9)
    best_of_10["num_full_fwd"] = (best_of_10["fwd"] == 1.0)
    best_of_10["num_reachable"] = (best_of_10["reachable"] == 1.0) # np.logical_and((best_of_10["fwd"] == 1.0).values, (best_of_10["reachable"] == 1.0).values)
    best_of_10["num_trafficIsolation"] = (best_of_10["trafficIsolation"] == 1.0) # np.logical_and((best_of_10["fwd"] == 1.0).values, (best_of_10["reachable"] == 1.0).values)
    res = best_of_10.groupby(["num_nodes", "num_networks"]).mean()
    res["num_full_match"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_full_match"]
    res["num_full_fwd"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_full_fwd"]
    res["num_reachable"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_reachable"]
    res["num_trafficIsolation"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_trafficIsolation"]
    res["num_close_match"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_close_match"]
    res["num_close_match90"] = best_of_10.groupby(["num_nodes","num_networks"]).sum()["num_close_match90"]
    return res

In [2]:
def aggregate_paper(df, consistency_column="overall", aggregate_by_n=False):
    best_of_10 = df.groupby(["sample_id"]).max()
    best_of_10["num_full_match"] = (best_of_10["overall"] == 1.0)
    best_of_10["num_close_match"] = (best_of_10["overall"] > 0.95)
    best_of_10["num_close_match90"] = (best_of_10["overall"] > 0.9)
    best_of_10["num_full_fwd"] = (best_of_10["fwd"] == 1.0)
    best_of_10["num_reachable"] = (best_of_10["reachable"] == 1.0) # np.logical_and((best_of_10["fwd"] == 1.0).values, (best_of_10["reachable"] == 1.0).values)
    best_of_10["num_trafficIsolation"] = (best_of_10["trafficIsolation"] == 1.0) # np.logical_and((best_of_10["fwd"] == 1.0).values, (best_of_10["reachable"] == 1.0).values)
    
    num_nodes_values = list(sorted(best_of_10["num_nodes"].values))
    if len(num_nodes_values) / 3 != int(len(num_nodes_values) / 3):
        print("warning: provided samples cannot be evenly distributed into 3 datasets small/medium/large")
    first_boundary = int(len(num_nodes_values) / 3)
    second_boundary = int(len(num_nodes_values) * 2 / 3)
    # print("first_boundary:", num_nodes_values[first_boundary - 1])
    # print("second_boundary:", num_nodes_values[second_boundary - 1])
    # print("third_boundary:", num_nodes_values[-1])
    # print(num_nodes_values)

    best_of_10["group"] = pd.cut(best_of_10["num_nodes"], [0,18,39,153])
    assert all([v == 8 for v in best_of_10.groupby("group").count()["overall"].values]), "The provided samples cannot distributed into three datasets of equal size 8 with num_nodes boundaries [0, 18, 39, 153]: Instead got num_nodes values {}".format(num_nodes_values)
    
    res = best_of_10.groupby("group").mean()
    std = best_of_10.groupby("group").std(ddof=0)

    for f in ["overall"]:
        res[f + "-std"] = std[f] 

    res["num_full_match"] = best_of_10.groupby("group").sum()["num_full_match"]
    res["num_full_fwd"] = best_of_10.groupby("group").sum()["num_full_fwd"]
    res["num_reachable"] = best_of_10.groupby("group").sum()["num_reachable"]
    res["num_trafficIsolation"] = best_of_10.groupby("group").sum()["num_trafficIsolation"]
    res["num_close_match"] = best_of_10.groupby("group").sum()["num_close_match"]
    res["num_close_match90"] = best_of_10.groupby("group").sum()["num_close_match90"]
    return res
    
def to_latex_paper(df):
    #print(to_latex(df))
    lines = []
    for row, group in zip(df.iloc, ["S", "M", "L"]):
        # def group_name(g): return "S" if g == 0 else ("M" if g == 1 else "L")
        # group = group_name(row["group"])
        columns = [
            group,
            "%.2f" % row["fwd"],
            "%.2f" % row["reachable"],
            "%.2f" % row["trafficIsolation"],
            "\\textbf{%.2f}" % row["overall"] + "$\pm$%.2f" % row["overall-std"],
            "%d/8" % row["num_full_match"],
            "%d/8" % row["num_close_match90"],
        ]
        lines.append("& " + " & ".join(columns) + "\\\\")
    return "\n".join(lines)

## Figure on average specification consistency for increasingly large topologies and specifications

### 3x2, EXP_ID=eval-64-bgp-reqs-2-4shot python3 eval_consistency.py ../../trained-model/bgp-64-pred-6layers-model-epoch2800.pt --num-samples=5 --random=1 --num-shots=4 --num-iterations=6 --dataset ./dataset-ported/bgp-qlty-reqs-2 --cpu=1

In [3]:
f = "specification-consistency/results-eval-64-bgp-qlty-reqs-2-4shot-15649.pkl-results.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print("$3\\times 2$ " + to_latex_paper(df_2))
print("\\midrule")

$3\times 2$ & S & 0.97 & 0.94 & 1.00 & \textbf{0.96}$\pm$0.07 & 6/8 & 6/8\\
& M & 0.95 & 0.94 & 1.00 & \textbf{0.94}$\pm$0.08 & 5/8 & 5/8\\
& L & 0.92 & 1.00 & 1.00 & \textbf{0.94}$\pm$0.06 & 4/8 & 4/8\\
\midrule


### 3x8, EXP_ID=eval-64-bgp-reqs-8-4shot python3 eval_consistency.py ../../trained-model/bgp-64-pred-6layers-model-epoch2800.pt --num-samples=5 --random=1 --num-shots=4 --num-iterations=6 --dataset ./dataset-ported/bgp-qlty-reqs-8 --cpu=1

In [4]:
f = "specification-consistency/results-eval-64-bgp-qlty-reqs-8-4shot-14793.pkl-results.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print("$3\\times 8$ " + to_latex_paper(df_2))
print("\\midrule")

$3\times 8$ & S & 0.98 & 0.98 & 0.91 & \textbf{0.96}$\pm$0.05 & 4/8 & 7/8\\
& M & 0.97 & 0.98 & 1.00 & \textbf{0.98}$\pm$0.03 & 4/8 & 8/8\\
& L & 0.96 & 0.92 & 0.97 & \textbf{0.95}$\pm$0.03 & 1/8 & 8/8\\
\midrule


### 3x16, EXP_ID=eval-64-bgp-reqs-16-4shot python3 eval_consistency.py ../../trained-model/bgp-64-pred-6layers-model-epoch2800.pt --num-samples=5 --random=1 --num-shots=4 --num-iterations=6 --dataset ./dataset-ported/bgp-qlty-reqs-16 --cpu=1

In [5]:
f = "specification-consistency/results-eval-64-bgp-qlty-reqs-16-4shot-11562.pkl-results.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print("$3\\times 16$ " + to_latex_paper(df_2))

$3\times 16$ & S & 0.98 & 0.92 & 0.95 & \textbf{0.95}$\pm$0.03 & 2/8 & 8/8\\
& M & 0.95 & 0.95 & 0.98 & \textbf{0.96}$\pm$0.04 & 3/8 & 7/8\\
& L & 0.94 & 0.91 & 0.95 & \textbf{0.93}$\pm$0.05 & 1/8 & 6/8\\


## Figure on Baseline/Oneshot/Multi-Shot Comparison

In [6]:
def to_latex_paper_baseline(df):
    #print(to_latex(df))
    lines = []
    for row in df.iloc:
        def group_name(g): return "Small" if g == 0 else ("Medium" if g == 1 else "Large")
        group = group_name(row["num_nodes"])
        columns = [
            group,
            "%.2f" % row["overall"] + "$\pm$%.2f" % row["overall-std"],
        ]
        lines.append("& " + " & ".join(columns) + "\\\\")
    return "\n".join(lines)

In [7]:
f = "multishot-sampling/results-eval-64-bgp-qlty-reqs-16-random-13819.pkl-results.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print("Random")
print(to_latex_paper_baseline(df_2))

print("1-Shot")
f = "multishot-sampling/results-eval-64-bgp-qlty-reqs-16-oneshot-14551.pkl-results.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print(to_latex_paper_baseline(df_2))

print("4-Shot")
f = "specification-consistency/results-eval-64-bgp-qlty-reqs-16-4shot-11562.pkl-results.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print(to_latex_paper_baseline(df_2))

Random
& Large & 0.87$\pm$0.11\\
& Large & 0.81$\pm$0.10\\
& Large & 0.80$\pm$0.05\\
1-Shot
& Large & 0.94$\pm$0.04\\
& Large & 0.96$\pm$0.04\\
& Large & 0.93$\pm$0.05\\
4-Shot
& Large & 0.95$\pm$0.03\\
& Large & 0.96$\pm$0.04\\
& Large & 0.93$\pm$0.05\\


In [9]:
print("8-Shot")
f = "multishot-sampling/results-eval-64-bgp-qlty-reqs-16-8shot.pkl"
df_2 = aggregate_paper(pd.read_pickle(f)).reset_index()
print(to_latex_paper_baseline(df_2))

8-Shot
& Large & 0.95$\pm$0.04\\
& Large & 0.96$\pm$0.04\\
& Large & 0.94$\pm$0.05\\
